# Building a minimal ReAct AI agent

The goal is to build a simple LangChain-powered agent that:
- Takes a user query
- Decides what tool to use (calculator or web search)
- Calls the tool
- Returns an answer

We'll use 
- Langchain
- Gemini (via ```google.generativeai```)
- tool (Calculator)
- ReAct-style logic (Reason + action + Observation)

## End-to-End agent

## Task 1: Imports and instantiate the central model

In [3]:
# imports
from google import genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from load_env import gemini_api_key
from langchain_core.messages import HumanMessage
from IPython.display import Markdown, display

# Create a model instance
llm = ChatGoogleGenerativeAI(
    model = 'gemini-2.0-flash',
    api_key = gemini_api_key,
    temperature = 0.3,
    max_tokens = 1500,
    timeout = None,
    max_retries = 2,
    # some other parameters
)

# Test the model
response = llm.invoke('Hello! What is Langchain?')
display(Markdown(response.content))

Langchain is a framework designed to simplify the development of applications powered by large language models (LLMs). Think of it as a toolkit that helps you connect LLMs to other data sources and tools, allowing you to build more sophisticated and useful applications.

Here's a breakdown of what it is and why it's useful:

**Core Concepts:**

* **LLMs (Large Language Models):**  These are the powerful AI models like GPT-3, GPT-4, PaLM, Llama 2, etc., that can generate text, translate languages, write different kinds of creative content, and answer your questions in an informative way. Langchain helps you work with these models.

* **Components:** Langchain provides modular components that you can assemble to create different types of applications.  These components include:
    * **Models:**  Wrappers around different LLMs, making them easy to use within Langchain.
    * **Prompts:**  Templates and tools for crafting effective prompts to guide the LLM's behavior.  Prompt engineering is crucial for getting the desired output.
    * **Chains:**  Sequences of calls to LLMs or other utilities.  This allows you to create complex workflows, like summarizing a document and then answering questions about it.
    * **Indexes:**  Ways to structure documents so that LLMs can easily retrieve relevant information from them.  This is essential for working with large datasets.
    * **Memory:**  Allows chains and agents to remember previous interactions, enabling conversational applications.
    * **Agents:**  Use an LLM to decide which actions to take, allowing them to interact with the outside world (e.g., using search engines, calculators, or other APIs).
    * **Callbacks:**  Hooks that allow you to inspect and modify the execution of chains and agents.  Useful for logging, debugging, and monitoring.

**Why is Langchain Useful?**

* **Abstraction:**  It abstracts away the complexities of working directly with LLM APIs.  You don't need to worry about the low-level details of authentication, rate limiting, or data formatting.

* **Modularity:**  The modular design allows you to easily swap out different components (e.g., different LLMs, different vector databases) without rewriting your entire application.

* **Composability:**  You can combine different components to create complex and powerful applications.

* **Standardization:**  Langchain provides a standardized way to interact with LLMs, making it easier to build applications that are portable and maintainable.

* **Community and Ecosystem:**  Langchain has a large and active community, which means there are plenty of resources, examples, and support available.  It also integrates with a wide range of other tools and services.

**What can you build with Langchain?**

* **Chatbots:**  Build conversational AI agents that can answer questions, provide support, or even just chat.
* **Question Answering Systems:**  Create systems that can answer questions based on a specific set of documents or data.
* **Summarization Tools:**  Automatically summarize long documents or articles.
* **Code Generation Tools:**  Generate code from natural language descriptions.
* **Data Analysis Tools:**  Use LLMs to analyze data and generate insights.
* **Agents that can take actions:**  Build agents that can use tools like search engines, calculators, and APIs to accomplish tasks.

**In summary, Langchain is a powerful framework that simplifies the process of building applications powered by large language models. It provides a set of tools and abstractions that make it easier to connect LLMs to other data sources and tools, allowing you to create more sophisticated and useful applications.**

## Task 2: Defining tools

In [31]:
from langchain.tools import tool
import math

# we want a function that takes a string and returns the resuly
@tool
def calculate(expression: str) -> str:
    """
    Evaluates a math expression using Python's math library.
    You can use functions like sin, cos, sqrt, log, and constants like pi.
    Example: math.sin(30 * pi / 180)
    """
    import math
    try:
        result = eval(expression, {'__builtin__': {}, 'math': math})
        return str(result)
    except Exception as e:
        return f"Error: {e}"

# Test the tool
print(calculate.invoke('2+2'))      

4


In [14]:
!pip install langchain-community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached numpy-2.3.1-cp313-cp313-macosx_14_0_x86_64.whl.metadata (62 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached frozenlist-1.7.0-cp313-cp313-macosx_10_13_x86_64.whl.metadata (18 kB)
  Using cached multidict-6.6.3-cp313-cp313-macosx_10_13_x86_64.whl.metadata (5.3 kB)
  Using cached propcache-0.3.2-cp313-cp313-macosx_10_13_x86_64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp313-cp313-macosx_10_13_x86_64.whl.metadata (73 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.0 MB/s eta 0:00:00
Using cached

In [16]:
!pip install -U duckduckgo-search --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [22]:
# let's set up a web search tool
# for a minimal demo, we can use a simple web search
# like DuckDuckGo

from langchain_community.tools import DuckDuckGoSearchRun

# create the search tool

search = DuckDuckGoSearchRun()

# Test the tool
response = search.invoke('Who won the Fifa Club world cup?')
from pprint import pprint
pprint(response)

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


('1 day ago · Clubs from across the globe have competed in the FIFA Club World '
 'Cup over the last 25 years, with the latest edition featuring giants from '
 'all six of world soccer’s governing … 1 day ago · Chelsea beat Paris '
 'Saint-Germain 3-0 at the FIFA Club World Cup final in front of U.S. '
 'President Donald Trump. See all the goals and highlights. Jun 23, 2025 · A '
 'complete record of FIFA Club World Cup winners since 2000, highlighting '
 'major format changes and key moments in tournament history. 21 hours ago · '
 'The FIFA Club World Cup expanded to a 32-team quadrennial event in 2025. '
 'Real Madrid has won the tournament a record five times, claiming victory in '
 '2014, 2016, 2017, 2018 … 1 day ago · Chelsea have won the FIFA Club World '
 'Cup after an impressive 3-0 victory over UEFA Champions League winner Paris '
 'Saint-Germain in the final in New Jersey. The Blues were brilliant …')


## Task 3: Define the Agent's Prompt and Tool list

In [23]:
tools = [calculate, search]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    (
        'system', 
        'You are a helpful AI assistant. You have access to the following tools: calculator, web search. Use them first of all to answer user questions. Think step by step.'
    ),
    ('user', '{input}')
])

chain = prompt | llm #do chain prompting

response = chain.invoke({'input': 'What is the square root of 144?'})
response # this response is not good, it is not using the tools

AIMessage(content="Here's how to find the square root of 144:\n\n*   **Understanding Square Root:** The square root of a number is a value that, when multiplied by itself, equals the original number.\n\n*   **Calculation:** The square root of 144 is 12 because 12 * 12 = 144.\n\n**Answer:** The square root of 144 is 12.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--5a54d352-ad74-4a4a-bf30-082b559026e1-0', usage_metadata={'input_tokens': 46, 'output_tokens': 93, 'total_tokens': 139, 'input_token_details': {'cache_read': 0}})

In [32]:
# import the agent class
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)


In [ ]:
# invoke the agent
response = agent.invoke('What is the square root of 144?')
print(response)



> Entering new AgentExecutor chain...


I need to calculate the square root of 144. I can use the calculate tool for this.
Action: calculate
Action Input: sqrt(144)
Observation: Error: name 'sqrt' is not defined
Thought:I made a mistake by assuming the `calculate` tool knows the `sqrt` function. I should instead use the power operator to calculate the square root.
Action: calculate
Action Input: 144**0.5
Observation: 12.0
Thought:I now know the final answer
Final Answer: 12.0

> Finished chain.
{'input': 'What is the square root of 144? You can use math library for the calculate tool', 'output': '12.0'}


In [30]:
response = agent.invoke('What is the sin of 30 degrees?')
print(response)



> Entering new AgentExecutor chain...
I know that the sine of 30 degrees is a common trigonometric value.
Action: calculate
Action Input: sin(30)
Observation: Error: name 'sin' is not defined
Thought:I need to use a calculator that understands trigonometric functions. I'll try expressing 30 degrees in radians and then using the math library.
Action: calculate
Action Input: sin(30 * pi / 180)
Observation: Error: name 'sin' is not defined
Thought:I need to use a calculator that understands trigonometric functions. I will try to use duckduckgo search to find the answer.
Action: duckduckgo_search
Action Input: sin(30 degrees)

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Nov 22, 2024 · To determine the sine and cosine of 30 degrees, we can use the special values known in trigonometry for this angle. Sine of 30°: The sine function represents the ratio of the … Mar 16, 2025 · Sin (30) = 0.5 All the trig. values are based on a triangle integrated into a circle. Practically, On a piece of graph paper, draw a circle centred on the origin (0,0). Call the radius of … Aug 11, 2024 · What is the value of sin 30 degrees? The value of \sin 30^\circ (sine of 30 degrees) is a fundamental trigonometric value that is commonly used in various fields of mathematics and … Oct 22, 2018 · Answer: the derivative of sin 30 degrees will be zero. Step-by-step explanation: First of all, we need to know what is the value of sin 30 degrees. Step 1:- According to the … May 23, 2018 · Find an answer to your question Find the value of sin 30 degree geometrically.
Thought:The search results indicate that sin(30 degrees) = 0.5
Final Answer: 0.5

> Finished chain.
{'inpu

In [34]:
response = agent.invoke('What is the sin of 3.14 radians??')
response



> Entering new AgentExecutor chain...
I need to calculate the sine of 3.14 radians. I can use the calculate tool for this.
Action: calculate
Action Input: sin(3.14)
Observation: Error: name 'sin' is not defined
Thought:I need to calculate the sine of 3.14 radians. I need to use the calculate tool, but it seems it doesn't recognize the sin function. I will try using the math.sin function.
Action: calculate
Action Input: math.sin(3.14)
Observation: 0.0015926529164868282
Thought:I have calculated the sine of 3.14 radians.
Final Answer: 0.0015926529164868282

> Finished chain.


{'input': 'What is the sin of 3.14 radians??',
 'output': '0.0015926529164868282'}

## Task 4: Test and Refine your agent
Test your agent with a variety of queries to ensure it uses the right tool and handles edge cases. Refine tool docstrings or logic as needed


In [37]:
## math queries
queries = [
    'What is 12 multiplied by 15?',
    'What is the square of 144, raised to the power of 1/3?. Break the calculations into steps',
    'what is tan(sin(75)). Break the calculation to help?'
]
for query in queries:
    print(agent.invoke(query))



> Entering new AgentExecutor chain...


I need to calculate the product of 12 and 15.
Action: calculate
Action Input: 12 * 15
Observation: 180
Thought:I now know the final answer.
Final Answer: 180

> Finished chain.
{'input': 'What is 12 multiplied by 15?', 'output': '180'}


> Entering new AgentExecutor chain...
First, I need to calculate the square of 144.
Action: calculate
Action Input: 144*144
Observation: 20736
Thought:

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 42
}
].


Now I need to raise the result (20736) to the power of 1/3, which is the same as taking the cube root.
Action: calculate
Action Input: 20736**(1/3)
Observation: 27.47314182127996
Thought:

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 40
}
]

In [38]:
queries = [
    'Who is the president of France?',
    'What is the capital of Tibet?'
]

for query in queries:
    print(agent.invoke(query))



> Entering new AgentExecutor chain...
I need to find out who the current president of France is. I can use a search engine to find this information.
Action: duckduckgo_search
Action Input: "president of France"

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: 1 day ago · Emmanuel Macron is a French banker and politician who was elected president of France in 2017. Macron was the first person in the history of the Fifth Republic to win the … Dec 5, 2024 · Emmanuel Macron vows to stay in office until 2027 and names a new prime minister after a historic no-confidence vote at the National Assembly. He blames his opponents … Dec 5, 2024 · French President Emmanuel Macron faces the challenge to pick a prime minister capable of leading a minority government in a parliament where no party holds a majority. Dec 6, 2024 · French President Emmanuel Macron addresses the nation following the resignation of ousted Prime Minister Michel Barnier, a day after a historic no-confidence vote at the … 6 days ago · Emmanuel Macron is on a three-day state visit to the UK, the first by a French president since 2008. Watch live below as he addresses parliament - after a carriage …
Thought:The search results indicate that Emmanuel Macron is the current

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Dec 20, 2022 · Lhasa is the capital city of Tibet before 1959. It was the administrative center of Tibet and Gaden Podrang was the name of the government. After Chinese invasion, Tibetan … No, the capital of Tibet is Lhasa. Nepal is a country not a capital. May 3, 2024 · Canberra is the capital city of Australia. the capital is Berlin. Rome. Rome is the capital of Italy and is nicknamed the Eternal City. It runs through the eastern banks of the river … Oct 6, 2023 · One View:The common people in Tibet eat mutton, beef and meat of yak. Some of the Buddhists prefer vegetarian food.Tibetans have rice, wheat and barley as staple foods, … The capital city of Tibet is Lhasa.
Thought:I have found the answer.
Final Answer: Lhasa

> Finished chain.
{'input': 'What is the capital of Tibet?', 'output': 'Lhasa'}


In [39]:
# Mixed
queries = [
    'What is the population of Germany divided by 2? Think in steps',
    'What is the cosine of the current temperature in New york?'
]

for query in queries:
    print(agent.invoke(query))  



> Entering new AgentExecutor chain...
I need to find the population of Germany first, then divide it by 2.
Action: duckduckgo_search
Action Input: population of Germany

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Feb 16, 2025 · The estimated population of Germany in 1941 was around 70 million people. Apr 26, 2024 · In the 1930s, the population of Germany was approximately 67 million people. Oct 27, 2022 · In 1939 the population of Germany (including annexed Austria, Memelland, and the Sudetenland) was … Sep 1, 2023 · In 1914 Germany had a population of 67 million. Berlin was the capital and Kaiser Wilhelm II was the … Feb 16, 2025 · The population of Germany in 1941 was approximately 69 million.
Thought:I need to find the current population of Germany.
Action: duckduckgo_search
Action Input: current population of Germany

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Aug 7, 2018 · You mention over due bills. If your missing credits follow late payments, that’s your answer. Pay EVERY bill on time without fail and the problem may resolve. … Oct 31, 2020 · my current router, 2Wire 3801 HGV-B, it's OLD and I NEED A BETTER REPLACEMENT! HOW CAN I GET A NEW MODEL? has an issue with intermittently … May 22, 2018 · Just move the SIM card from your current phone to the new phone. If the new phone takes a different size, go to an ATT store for a replacement, they will activate it … Aug 13, 2018 · When reviewing usage logs for "Current Billed Usage (Jun xx, 2018- Jul yy,2018), the itemized text log shows the occurrences correctly with regards to … Sep 18, 2017 · Is there a way an agent or representative can check 10 days of current billing cycle to turn in for business expense …
Thought:I need to find the current population of Germany.
Action: duckduckgo_search
Action Input: current population of Germany

...


Observation: Check current conditions in New York, NY with radar, hourly, and more. Get New York, NY current weather report with temperature, feels like, wind, humidity, pressure, UV and more from TheWeatherNetwork.com. Today’s and tonight’s Manhattan, NY weather forecast, weather conditions and Doppler radar from The Weather Channel and Weather.com 4 days ago · Currently: 87 °F. Passing clouds. (Weather station: New York City - Central Park, USA). See more current weather 1 day ago · Tomorrow's temperature is forecast to be NEARLY THE SAME as today. New York City Weather Forecasts. Weather Underground provides local & long-range weather forecasts, …
Thought:The current temperature in New York City is 87°F. Now I need to calculate the cosine of 87. Since the cosine function typically takes radians as input, I will assume the question is asking for the cosine of 87 radians.

Action: calculate
Action Input: cos(87)
Observation: Error: name 'cos' is not defined
Thought:The calculator doe

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Inverse cosine calculator -1 = Calculate Reset Degrees First result Second result Use this cos calculator to easily calculate the cosine of an angle given in degrees or radians. The cosine is encountered in … Effortlessly find trigonometric function values (sin, cos, tan, cot) or solve for missing sides or angles in a right … The online free cosine calculator helps in calculating the cosine value of the given angle in degrees, radians, mili-radians, … Use the cos calculator to find the cosine of an angle in radians or degrees, plus learn the trigonometric formulas and …
Thought:I will use the value I calculated earlier (1.5184364492351667 radians) and input it into the cosine calculator I found online. Since I can't directly interact with the online calculator, I will use the calculate tool with the assumption that it can handle the cosine function if I input the radian value directly.

Action: calculate
Action Input: cos(1.5184364492351667)
Observation: Error: name 'cos'

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 26
}
].


I am still unable to calculate the cosine. It seems the calculate tool is very limited. I will try to approximate the cosine using a Taylor series expansion. The cosine of x is approximately 1 - x^2/2 + x^4/24.

Action: calculate
Action Input: 1 - (1.5184364492351667)**2 / 2 + (1.5184364492351667)**4 / 24
Observation: 0.06867614422175114
Thought:I have approximated the cosine of the temperature in radians.

Final Answer: 0.06867614422175114

> Finished chain.
{'input': 'What is the cosine of the current temperature in New york?', 'output': '0.06867614422175114'}


In [40]:
agent.invoke('My height in cm is 165cm. what is my height in yards?')



> Entering new AgentExecutor chain...
I need to convert centimeters to yards. I can use the search tool to find the conversion factor.
Action: duckduckgo_search
Action Input: convert cm to yards

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: No good DuckDuckGo Search Result was found
Thought:I need to convert centimeters to yards. I can use the search tool to find the conversion factor.
Action: duckduckgo_search
Action Input: cm to yards conversion factor

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Instant free online tool for centimeter to inch conversion or vice versa. The centimeter [cm] to inch [in] conversion table and conversion steps are also listed. Also, explore tools to convert centimeter or inch to other length … Jan 27, 2023 · Convert quickly and easily between centimeters (cm) and inches (in) using our conversion tool. Our converter … Do a quick conversion: 1 centimetres = 0.39370078740157 inches using the online calculator for metric conversions. Check the chart for more details. To convert a measurement in centimeters to inches, simply divide the number of centimeters by 2.54. For example, if you have a measurement of 50 centimeters, you would divide 50 by 2.54 to get the equivalent … A centimetre (International spelling) or centimeter (American English), with SI symbol cm, is a unit of length in the International System of Units (SI) equal to one hundredth of a metre, centi- being the SI prefix for a …
Thought:I need to convert cm to yards. The searc

{'input': 'My height in cm is 165cm. what is my height in yards?',
 'output': '1.8044583333333333'}

In [41]:
import time
def safe_agent_invoke(agent, query, tries = 0):
    if tries > 3:
        print(f"Failed to invoke agent after {tries} attempts")
        return None
    try:
        return agent.invoke(query)
    except Exception as e:
        time.sleep(3)
        return safe_agent_invoke(agent, query, tries + 1)

queries = [
    'What is the population of Germany divided by 2? Think in steps',
    'What is the cosine of the current temperature in New york?'
]

for query in queries:
    response = safe_agent_invoke(agent, query)
    print(response)



> Entering new AgentExecutor chain...
I need to find the population of Germany first, then divide it by 2.
Action: duckduckgo_search
Action Input: population of Germany

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: As of 2025, the population of Germany is projected at 84.08 million, compared to 84.55 million in 2024. It is the second-most populous country in Europe and ranks 19th in the world. 5 days ago · The current population of Germany is 84,060,105 with an annual growth rate of -0.794%. Germany has a population of 1.03% of the world's population. The net migration rate of … Jun 20, 2025 · As of the end of 2024, Germany's population stood at just under 83.6 million, an increase of 121,000 compared to the previous year. This marks a slowdown from 2023, when the … Jan 10, 2025 · Table with data on: Population by nationality and sex 1970 to 2023 in Germany. Apr 19, 2025 · On 01 Jul 2025 Germany population will be 80682351 people. It’s -0.01% (6194 people) compared with the population of Germany on 01 Jul 2024. These data (statistics and …
Thought:I have several estimates for the population of Germany. I will use the most recent estimate, which is 84,060,105. Now I need to divide th

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: 3 days ago · Current conditions at New York City, Central Park (KNYC) Lat: 40.78°N Lon: 73.97°W Elev: 154ft. 2 days ago · New York, New York - Current temperature and weather conditions. Detailed hourly weather forecast for today - including weather conditions, temperature, pressure, humidity, … 1 day ago · Live New York City, NY weather with granular precipitation, radar, barometric pressure, temperatures and wind speeds for today thru Thursday, as you toggle through each … 3 days ago · The weather today in New York City will be hot with temperatures reaching 86°F. During the evening and night time the temperatures will drop to 71 ° F. For deep dive … Jul 4, 2025 · Get New York, NY weather with accurate real-time conditions, 7-day outlooks, and hourly updates. Plan outdoor activities with temperature, precipitation, wind, and severe …
Thought:The current temperature in New York City is approximately 86°F. Now I need to calculate the cosine of 86.
Action: calculate
Action

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Jun 24, 2025 · Here are some tips to help you get the right answer using the Cosine Calculator. Understand the Function: The cosine of an angle is defined as the ratio of the adjacent side to … Jun 10, 2025 · Calculate inverse cosine (arccos) values instantly. Enter any value between -1 and 1 to get results in degrees or radians with detailed explanations. Sep 18, 2024 · Converting degrees to radians is a fundamental skill in mathematics, particularly in trigonometry, calculus, and various fields like physics and engineering. This Degrees to Radian … Mar 30, 2025 · The cosine of pi (π) is -1. Here's how you can find that value: Using the Unit Circle The unit circle is a circle with a radius of 1 centered at the origin of a coordinate plane. Angles … 6 days ago · Explain why cos θ ≤ 1 cos ⁡ θ ≤ 1 for any angle θ θ. How many solutions are there to the equation cos(θ) = 0.4 cos ⁡ (θ) = 0.4 between 0∘ 0 ∘ and 360∘ 360 ∘? On a unit circle, the ___ …
Thought:I cannot directly c

/Users/tenzinjampa/Downloads/personal_projects/AI-agent-grind/venv/lib/python3.13/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: No good DuckDuckGo Search Result was found
Thought:I cannot directly calculate the cosine of a value using the available tools. I can use the search engine to find the answer. Since the search engine did not return a direct answer, I will try to calculate the cosine using an online calculator and then provide the answer.

Since I cannot perform the calculation with the available tools, I will have to provide an estimated answer. The current temperature in New York is approximately 86 degrees Fahrenheit. Converting this to radians gives approximately 1.5 radians. The cosine of 1.5 radians is approximately 0.07.

Final Answer: Approximately 0.07

> Finished chain.
{'input': 'What is the cosine of the current temperature in New york?', 'output': 'Approximately 0.07'}
